## BETTER WAY 17 - 인수를 순회할 때는 방어적으로 하자
## Item17 - Be Defensive When Iterating Over Arguments

파라미터로 객체의 리스트를 받는 함수에서 리스트를 여러 번 순회해야 할 때가 간혹 발생한다. 예를 들어 서울에 방문한 여행자 수를 분석하고 싶다고 하자. 데이터 집합은 각 구의 방문자 수(연도별 백만 명 단위)라고 하면, 각 구에서 전체 여행자 중 몇 퍼센트인지 구하는 코드는 아래와 같다.

In [1]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [2]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


이제 파일에서 데이터를 읽어와 위의 작업을 수행하는 제너레이터를 만들어 보자.

In [3]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [4]:
it = read_visits('./examples/numbers.txt')
percentages = normalize(it)  # <-- normalize 함수를 호출하면서 이터레이터가 사용됨
print(percentages)

[]


위의 결과를 보면 아무것도 출력 되지 않는다. 그 이유는 이터레이터 `it`이  결과를 한 번만 생성하기 때문이다. 아래의 코드를 보면 확실히 알 수 있다. 

In [5]:
it = read_visits('./examples/numbers.txt')
print(list(it))  # <- 여기서 사용됨
print(list(it))  # <- 빈값을 출력

[100, 11, 123, 122, 166, 188, 298, 300, 345, 432, 315, 462, 176, 542]
[]


이 문제를 해결하려면 이터레이터의 복사본을 리스트에 저장해야 한다. 

In [6]:
def normalize_copy(numbers):
    numbers = list(numbers)  # 이터레이터를 복사함
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [7]:
it = read_visits('./examples/numbers.txt')
percentages = normalize_copy(it)
print(percentages)

[2.793296089385475, 0.30726256983240224, 3.435754189944134, 3.4078212290502794, 4.636871508379889, 5.251396648044692, 8.324022346368714, 8.379888268156424, 9.636871508379889, 12.067039106145252, 8.798882681564246, 12.905027932960893, 4.916201117318436, 15.139664804469273]


위의 `normalize_copy`함수의 문제는 이터레이터의 복사본이 클 수도 있다는 점이다. 그럴 경우 메모리를 많이 잡아 먹어 동작이 멈출 수도 있다. 이러한 문제를 해결하는 방법 중 하나는 호출될 때마다 새 이터레이터를 반환하는 함수를 받게 만드는 것이다.

In [8]:
def normalize_func(get_iter):
    total = sum(get_iter())  # 새 이터레이터 
    result = []
    for value in get_iter():  # 새 이터레이터
        percent = 100 * value / total
        result.append(percent)
    return result

`normalize_func`을 사용하려면 제너레이터(여기서는 `read_visits`)를 호출해서 매번 새 이터레이터를 생성하는 람다 표현식을 넘겨주면 된다.

In [9]:
path = './examples/numbers.txt'
pertanges = normalize_func(lambda: read_visits(path))
print(percentages)

[2.793296089385475, 0.30726256983240224, 3.435754189944134, 3.4078212290502794, 4.636871508379889, 5.251396648044692, 8.324022346368714, 8.379888268156424, 9.636871508379889, 12.067039106145252, 8.798882681564246, 12.905027932960893, 4.916201117318436, 15.139664804469273]


코드가 잘 동작하지만, 람다 함수를 넘겨주는 방법은 세련되지 못하다(세련된거 같은 느낌적인 느낌...) 같은결과를 얻는 더 좋은 방법은 이터레이터 프로토콜(iterator protocol)을 구현한 새 컨테이너 클래스를 만드는 것이다. <br />
이터레이터 프로토콜은 파이썬의 `for`루프와 관련 표현식이 컨테이너 타입의 콘텐츠를 탐색하는 방법이다. 무슨 설명인지 이해가 가질 않는다.. 아래는 교재에 설명이 되어있는 부분이다.
> 파이썬은 `for x in foo` 같은 문장을 만나면 실제로는 `iter(foo)`를 호출한다. 그러면 내장 함수 iter는 특별한 메서드인 `foo.__iter__`를 호출한다.` __iter__` 메서드는 (`__next__`라는 특별한 메서드를 구현하는) 이터레이터 객체를 반환해야 한다. 마지막으로 `for` 루프는 이터레이터를 모두 소진할 때까지 (그래서 StopIteration 예외가 발생할 때까지) 이터레이터 객체에 내장 함수 `next`를 계속 호출한다. 
> *- 출처: 파이썬 코딩의 기술 -*

위의 설명은 어려운 듯 하지만 클래스에서 `__iter__` 메소드를 제너레이터로 구현해 주면 된다. 아래의 코드는 파일을 읽는 iterable (순회가능한) 컨테이너 클래스이다.

In [10]:
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

`ReadVisits`클래스를 이용하면 `normalize_copy`나 `normalize_func`과 같이 이터레이터를 복사해 주거나 매번 새 이터레이터를 생성해주는 방법을 쓰지 않아도 된다. 따라서 맨 위에 있는 `normalize` 함수를 사용해도 작동한다.

In [11]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [12]:
path = './examples/numbers.txt'
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)

[2.793296089385475, 0.30726256983240224, 3.435754189944134, 3.4078212290502794, 4.636871508379889, 5.251396648044692, 8.324022346368714, 8.379888268156424, 9.636871508379889, 12.067039106145252, 8.798882681564246, 12.905027932960893, 4.916201117318436, 15.139664804469273]


위의 코드가 작동하는 이유는 `normalize`의 `sum`메소드가 새 이터레이터 객체를 할당하기 위해 `ReadVisits.__iter__`를 호출하기 때문이다. 또한 `normalize`안의 `for`루프 또한 두 번째 이터레이터 객체를 할당할 때 `ReadVisits.__iter__`를 호출한다. 두 이터레이터는 독립적으로 동작하기 때문에 데이터 값을 얻을 수 있다. 이 방법의 단점은 입력 데이터를 여러번 읽는다는 점이다.

In [13]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError('리스트 형태의 데이터를 입력해 주세요')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [14]:
visits = [15, 35, 80]
normalize_defensive(visits)

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [15]:
visits = ReadVisits(path)
print(type(visits))
normalize_defensive(visits)

<class '__main__.ReadVisits'>


[2.793296089385475,
 0.30726256983240224,
 3.435754189944134,
 3.4078212290502794,
 4.636871508379889,
 5.251396648044692,
 8.324022346368714,
 8.379888268156424,
 9.636871508379889,
 12.067039106145252,
 8.798882681564246,
 12.905027932960893,
 4.916201117318436,
 15.139664804469273]

In [16]:
it = iter(visits)
normalize_defensive(it)

TypeError: 리스트 형태의 데이터를 입력해 주세요

### 정리
* 입력 인수를 여러 번 순회하는 함수를 작성할 때 주의하자. 입력 인수가 이터레이터라면 값이 사라질 수 있다.
* 파이썬의 이터레이터 프로토콜은 컨테이너와 이터레이터가 내장함수 `iter, next, for` 및 관련 표현식과 상호작용하는 방법을 정의한다.
* `__iter__` 메소드를 제너레이터로 구현하면 자신만의 이터러블 컨테이너 타입을 쉽게 정의할 수 있다.